In [1]:
import nest_asyncio
nest_asyncio.apply()

import sys
sys.path.append("../")

from desci_sense.shared_functions.init import init_multi_chain_parser_config
from desci_sense.shared_functions.parsers.multi_chain_parser import MultiChainParser

from desci_sense.shared_functions.dataloaders import (
    scrape_post,
    convert_text_to_ref_post,
)

In [2]:
config = init_multi_chain_parser_config(llm_type="google/gemma-7b-it",
                                        post_process_type="combined")

In [3]:
multi_chain_parser = MultiChainParser(config)

2024-04-30 15:41:32.755 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:65 - Initializing MultiChainParser. PostProcessType=combined
2024-04-30 15:41:32.759 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:72 - Initializing post parsers...
2024-04-30 15:41:32.759 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'refs_tagger' 
2024-04-30 15:41:32.807 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'topics' 
2024-04-30 15:41:32.833 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'keywords' 
2024-04-30 15:41:32.859 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'hashtags' 


In [4]:
# example raw text posts
TEST_POST_TEXT_W_REF = """
I really liked this paper!
https://arxiv.org/abs/2402.04607
"""

TEST_POST_2_TEXT_W_REF = """
I really disagree with this paper!
https://arxiv.org/abs/2402.04607
"""

TEXT_POSTS = [TEST_POST_TEXT_W_REF, TEST_POST_2_TEXT_W_REF]

In [5]:
# convert raw text to RefPost format for input to the parser
inputs = [convert_text_to_ref_post(p) for p in TEXT_POSTS]

In [12]:
# or scrape posts by url
urls = [
        "https://mastodon.social/@psmaldino@qoto.org/111405098400404613",
        "https://mastodon.social/@UlrikeHahn@fediscience.org/111732713776994953",
        "https://mastodon.social/@ronent/111687038322549430",
    ]
posts = [scrape_post(url) for url in urls]

In [6]:
# batch process
results = multi_chain_parser.batch_process_ref_posts(inputs)

2024-04-24 15:41:10.622 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation:54 - fetching citoid data for: https://arxiv.org/abs/2402.04607
2024-04-24 15:41:12.813 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:231 - Processing 2 posts with parsers: ['refs_tagger', 'topics', 'keywords', 'hashtags']
2024-04-24 15:41:12.815 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:233 - Instantiating prompts...
2024-04-24 15:41:12.816 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:243 - Invoking parallel chain...
2024-04-24 15:41:16.924 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:253 - Post processing 2 results...
2024-04-24 15:41:16.926 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:265 - Done!


In [6]:
#
url = "https://twitter.com/mbauwens/status/1779543397528740338"
post = scrape_post(url)

In [7]:
# single process
result = multi_chain_parser.process_ref_post(post)

2024-04-30 15:42:45.141 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async:20 - target_url=https://icandecide.org/v-safe-data/
2024-04-30 15:42:45.164 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async:20 - target_url=https://icandecide.org/pfizer-documents/
2024-04-30 15:43:04.309 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:184 - Processing post with parsers: ['refs_tagger', 'topics', 'keywords', 'hashtags']
2024-04-30 15:43:04.310 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:186 - Instantiating prompts...
2024-04-30 15:43:04.311 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:191 - Invoking parallel chain...


In [8]:
result.topics

['sustainability',
 'software & hardware',
 'academia',
 'climate',
 'news',
 'health',
 'technology',
 'research']

In [16]:
result.filter_classification

<SciFilterClassfication.RESEARCH: 'research'>

In [18]:
result.reference_tagger

['recommendation', 'reading']

In [19]:
result.topics

['economics', 'research']